In [36]:
import pandas as pd

# Librerías usadas para estandarizar la información
from repositories import dicts, utils


#Librerías gráficas y de procesamiento matemático
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
import numpy as np

## Lectura de datos poblacionales con el propósito de identificar municipios con población  femenina cero que deben ser excluidos del análisis

In [37]:
df1=pd.read_csv("data/anexo-area-sexo-edad-proyecciones-poblacion-Municipal_1995-2004.csv")
df2=pd.read_csv("data/anexo-area-sexo-edad-proyecciones-poblacion-Municipal_2005-2017.csv")

### Filtro x población

Los municipios cuya población entre 1998 y 2004 fue cero se enumeran

In [38]:
df1=df1[df1["AÑO"]>=1998].copy()
df1["Total Mujeres"]=sum(df1[f"Mujeres_{i}"] for i in range(101))
pob_0=set(df1[df1["Total Mujeres"]==0].DPMP.unique())
list(df1[df1.DPMP.isin(pob_0)].MPIO.unique())

['Norosí',
 'Norcasia',
 'Guachené',
 'Sucre',
 'Villa Rica',
 'San José de Uré',
 'Tuchín',
 'Carmen del Darién',
 'Cértegui',
 'Medio Atrato',
 'Medio Baudó',
 'Medio San Juan',
 'Río Iró',
 'Río Quito',
 'Unión Panamericana',
 'Albania',
 'La Jagua del Pilar',
 'Algarrobo',
 'Concordia',
 'Nueva Granada',
 'Sabanas de San Ángel',
 'Santa Bárbara de Pinto',
 'Zapayán',
 'Zona Bananera',
 'El Peñol',
 'Nariño',
 'Coveñas']

In [39]:
len(df1[df1.DPMP.isin(pob_0)].MPIO.unique())

27

In [40]:
df1[df1.DPMP.isin(pob_0)].DPMP.astype(str).unique()

array(['13490', '17495', '19300', '19785', '19845', '23682', '23815',
       '27150', '27160', '27425', '27430', '27450', '27580', '27600',
       '27810', '44035', '44420', '47030', '47205', '47460', '47660',
       '47720', '47960', '47980', '52254', '52480', '70221'], dtype=object)

Un total de 27 municipios tenían una población feménina de cero entre 1998 y 2004. Laz razones de esto puede ser que, si bien aparece en la base de datos, el municipio no se había constituido oficialmente para ese periodo o simplemente que corresponde a un territorio no municipalizado, por ejemplo forestal, donde no habitan personas.

In [19]:
df2["Total Mujeres"]=sum(df2[f"Mujeres_{i}"] for i in range(101))
pob_1=set(df2[df2["Total Mujeres"]==0].DPMP.unique())
list(df2[df2.DPMP.isin(pob_1)].MPIO.unique())

['Norosí', 'Guachené', 'San José de Uré', 'Tuchín']

In [20]:
len(df2[df2.DPMP.isin(pob_1)].MPIO.unique())

4

## Microdatos mortalidad por cáncer de seno

Inicialmente se carga la base de datos

In [4]:
df=pd.DataFrame(pd.read_csv("Cancer_1998_2018.csv"))
df["codmunicipio"]=df.apply(lambda x: int(str(x.codptore).zfill(2)+str(x.codmunre).zfill(3)),axis=1)
df.to_csv("Mortalidad_1998_2018.csv")

In [5]:
df["ok"]=np.where(df["codmunre"]==999,1,0)

In [9]:
df=df.sort_values(by="ok",ascending=True).copy()

In [11]:
df.to_excel("output_files/Cancer_1998_2018.xlsx")

Se eliminan de la tabla los datos correspondientes a los municipios que no se consideran en el análisis

In [21]:
df=df[~df.codmunicipio.isin(pob_0)].copy()


,Unnamed: 0,ano,est_civil,gru_ed1,gru_ed2,nivel_edu,codptore,codmunre,area_res,seg_social,...,c_bas1,c_mcm1,cau_homol,ultcurfal,ocupacion,idpertet,idadmisalud,causa_667,codmunicipio,ok
0,376,1998.0,5,14,4,5,19,75,2.0,2,...,C509,C509,24,NaN,NaN,NaN,NaN,NaN,19075,0
31252,194825,2013.0,6,17,5,4,18,1,1.0,2,...,C509,NaN,24,11.0,SIN INFORMACION,Ninguno de las anteriores,NaN,NaN,18001,0
31253,194879,2013.0,5,20,5,3,8,1,1.0,1,...,C509,NaN,24,9.0,HOGAR,Ninguno de las anteriores,NaN,NaN,8001,0
31254,194883,2013.0,6,18,5,3,8,1,1.0,1,...,C509,NaN,24,9.0,HOGAR,Ninguno de las anteriores,NaN,NaN,8001,0
31255,195105,2013.0,6,23,6,14,41,1,1.0,1,...,C509,NaN,24,99.0,HOGAR,Ninguno de las anteriores,NaN,NaN,41001,0
31256,195118,2013.0,4,21,6,14,5,1,1.0,1,...,C509,NaN,24,99.0,HOGAR,Ninguno de las anteriores,NaN,NaN,5001,0
31257,195186,2013.0,6,20,5,2,23,1,2.0,2,...,C509,NaN,24,2.0,HOGAR,Ninguno de las anteriores,NaN,NaN,23001,0
31258,195217,2013.0,6,18,5,2,19,548,3.0,1,...,C509,NaN,24,5.0,HOGAR,Ninguno de las anteriores,NaN,NaN,19548,0
31259,195276,2013.0,6,20,5,3,11,1,1.0,1,...,C509,NaN,24,9.0,HOGAR,Ninguno de las anteriores,NaN,NaN,11001,0
31251,194092,2013.0,6,19,5,3,8,1,1.0,3,...,C509,NaN,24,9.0,HOGAR,Ninguno de las anteriores,NaN,NaN,8001,0


In [34]:
df

,Unnamed: 0,ano,est_civil,gru_ed1,gru_ed2,nivel_edu,codptore,codmunre,area_res,seg_social,...,c_bas1,c_mcm1,cau_homol,ultcurfal,ocupacion,idpertet,idadmisalud,causa_667,codmunicipio,ok
0,376,1998.0,5,14,4,5,19,75,2.0,2,...,C509,C509,24,NaN,NaN,NaN,NaN,NaN,19075,0
31252,194825,2013.0,6,17,5,4,18,1,1.0,2,...,C509,NaN,24,11.0,SIN INFORMACION,Ninguno de las anteriores,NaN,NaN,18001,0
31253,194879,2013.0,5,20,5,3,8,1,1.0,1,...,C509,NaN,24,9.0,HOGAR,Ninguno de las anteriores,NaN,NaN,8001,0
31254,194883,2013.0,6,18,5,3,8,1,1.0,1,...,C509,NaN,24,9.0,HOGAR,Ninguno de las anteriores,NaN,NaN,8001,0
31255,195105,2013.0,6,23,6,14,41,1,1.0,1,...,C509,NaN,24,99.0,HOGAR,Ninguno de las anteriores,NaN,NaN,41001,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6204,26071,2002.0,9,19,5,9,1,999,9.0,1,...,C509,NaN,24,NaN,NaN,NaN,NaN,NaN,1999,1
19621,28081,2009.0,4,25,6,13,9,999,9.0,1,...,C509,NaN,24,0.0,NaN,Ninguno de las anteriores,NaN,NaN,9999,1
35006,61523,2015.0,2,15,4,4,9,999,9.0,3,...,C509,NaN,24,99.0,NaN,Ninguno de las anteriores,NaN,NaN,9999,1
20805,129081,2009.0,5,20,5,2,25,999,9.0,1,...,C509,NaN,24,99.0,NaN,Ninguno de las anteriores,NaN,NaN,25999,1


## Consolidación de los resultados a nivel municipio/año para el periodo de estudio

### exclusión de los datos que no tienen información sobre el municipio

In [22]:
df_try=df.copy()
len(df_try.codmunicipio.unique())


1047

Lectura de nombre de departamentos desde la librería de diccionarios

In [23]:
df_try.ano=df_try.ano.astype(int)

newdict={j:i for i,j in zip(dicts.DPTOS.keys(),dicts.DPTOS.values())}
newdict



{68: 'Santander',
 54: 'Norte de Santander',
 47: 'Magdalena',
 20: 'Cesar',
 81: 'Arauca',
 9: 'Sin información de departamento,',
 13: 'Bolivar',
 63: 'Quindio',
 17: 'Caldas',
 73: 'Tolima',
 11: 'Bogotá',
 66: 'Risaralda',
 25: 'Cundinamarca',
 76: 'Valle del Cauca',
 8: 'Atlántico',
 70: 'Sucre',
 44: 'La guajira',
 88: 'Archipiélago S.A, P. y Sta. C.',
 52: 'Nariño',
 19: 'Cauca',
 999: 'Con residencia en el extranjero.',
 86: 'Putumayo',
 27: 'Choco',
 5: 'Antioquia',
 23: 'Cordoba',
 50: 'Meta',
 99: 'Vichada',
 15: 'Boyaca',
 18: 'Caqueta',
 85: 'Casanare',
 41: 'Huila',
 95: 'Guaviare',
 91: 'Amazonas',
 94: 'Guainía',
 97: 'Vaupés'}

Se genera una tabla cruzada con el código del municipio y el año

In [24]:
df_try=pd.DataFrame(pd.crosstab(df_try.codmunicipio,df_try.ano)).reset_index()
df_try.codmunicipio=df_try.codmunicipio.apply(lambda x: str(x).zfill(5))
df_try

ano,codmunicipio,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,01999,6,3,5,0,1,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,05001,117,140,152,121,156,181,166,138,181,...,192,197,206,213,223,236,223,256,272,243
2,05002,0,0,1,0,0,0,1,2,0,...,0,1,0,0,2,1,1,4,1,0
3,05004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,1,0
4,05021,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,999604,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1043,999724,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1044,999840,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
1045,999862,0,0,0,0,0,0,0,0,0,...,1,0,3,0,0,2,0,1,1,5


### Utilizando información oficial sobre los nombres de los municipios se constuye uno para asociarlos a los códigos d elos municipios

In [28]:
dictio=pd.DataFrame(pd.read_csv("data/DIVIPOLA_Municipios.csv"))
dictio["MUNICIPIO"] = dictio["MUNICIPIO"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
dictio.COD=dictio.COD.astype(str)
dictio.COD=dictio.COD.apply(lambda x: x.zfill(5))
print(dictio.columns)
dictio=dictio.set_index('COD').T.to_dict()
for i in dictio.keys():
    dictio[i]=dictio[i]['MUNICIPIO'].lower()

print(dictio)


Index(['COD', 'MUNICIPIO'], dtype='object')
{'05001': 'medellin', '05002': 'abejorral', '05004': 'abriaqui', '05021': 'alejandria', '05030': 'amaga', '05031': 'amalfi', '05034': 'andes', '05036': 'angelopolis', '05038': 'angostura', '05040': 'anori', '05042': 'santa fe de antioquia', '05044': 'anza', '05045': 'apartado', '05051': 'arboletes', '05055': 'argelia', '05059': 'armenia', '05079': 'barbosa', '05086': 'belmira', '05088': 'bello', '05091': 'betania', '05093': 'betulia', '05101': 'ciudad bolivar', '05107': 'briceno', '05113': 'buritica', '05120': 'caceres', '05125': 'caicedo', '05129': 'caldas', '05134': 'campamento', '05138': 'canasgordas', '05142': 'caracoli', '05145': 'caramanta', '05147': 'carepa', '05148': 'el carmen de viboral', '05150': 'carolina', '05154': 'caucasia', '05172': 'chigorodo', '05190': 'cisneros', '05197': 'cocorna', '05206': 'concepcion', '05209': 'concordia', '05212': 'copacabana', '05234': 'dabeiba', '05237': 'donmatias', '05240': 'ebejico', '05250': 'el 

In [29]:
df_try["MUNICIPIO"]=df_try.codmunicipio.map(dictio)
df_try["codptore"]=df_try["codmunicipio"].apply(lambda x: x[:2])
df_try["DPTO"]=df_try["codptore"].astype(int).map(newdict)
df_try=df_try.fillna(0)

In [30]:
df_try.head(25)

ano,codmunicipio,1998,1999,2000,2001,2002,2003,2004,2005,2006,...,2012,2013,2014,2015,2016,2017,2018,MUNICIPIO,codptore,DPTO
0,01999,6,3,5,0,1,2,0,1,0,...,0,0,0,0,0,0,0,0,01,0
1,05001,117,140,152,121,156,181,166,138,181,...,213,223,236,223,256,272,243,medellin,05,Antioquia
2,05002,0,0,1,0,0,0,1,2,0,...,0,2,1,1,4,1,0,abejorral,05,Antioquia
3,05004,0,0,0,0,0,0,0,0,0,...,0,0,2,1,0,1,0,abriaqui,05,Antioquia
4,05021,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,alejandria,05,Antioquia
5,05030,0,1,2,0,0,1,0,1,1,...,1,1,2,1,3,3,1,amaga,05,Antioquia
6,05031,1,2,0,1,1,0,0,0,0,...,1,0,2,4,0,0,2,amalfi,05,Antioquia
7,05034,1,1,0,1,1,6,3,2,0,...,2,2,4,2,3,3,1,andes,05,Antioquia
8,05036,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,angelopolis,05,Antioquia
9,05038,0,0,0,1,3,0,0,0,0,...,0,0,1,0,0,0,0,angostura,05,Antioquia


# Los datos se exportan para su posterior uso

In [31]:
df_try.to_csv("output_files/CasosxMunicipio.csv")

In [32]:
len(df_try.codmunicipio.unique())

1047